# Week-5
This notebook is created during week-5 of MLzoomcamp - `Deploying Machine Learning Models`.

# Import modules

In [2]:
import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Load utilities

In [3]:
import utilities as utils

# Churn Prediction model from previous week

In [4]:
df = pd.read_csv('03_data-Telco-Customer-Churn.csv')

# Standardize column names 
df.columns = df.columns.str.lower().str.replace(' ','_')

# get list of categorical features
features_cat = list(df.dtypes[df.dtypes == 'object'].index)

# standardize categorical features data
for f in features_cat:
    df[f] = df[f].str.lower().str.replace(' ','_')
    
df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)


df.churn = (df.churn == 'yes').astype('int')

In [7]:
# Train-Test-Split (80-20)
df_train_full, df_test = utils.train_test_split(df, test_size=0.2, random_state=1)

In [8]:
# Numericals 
list_features_num = ['tenure', 'monthlycharges', 'totalcharges']

# Categoical 
list_features_cat = list(set(df_train_full.columns) - set(list_features_num) - set(['churn','customerid']))

# All Features 
list_features = list_features_num + list_features_cat
list_features

['tenure',
 'monthlycharges',
 'totalcharges',
 'paperlessbilling',
 'partner',
 'deviceprotection',
 'multiplelines',
 'gender',
 'paymentmethod',
 'streamingtv',
 'seniorcitizen',
 'streamingmovies',
 'techsupport',
 'internetservice',
 'contract',
 'onlinebackup',
 'phoneservice',
 'onlinesecurity',
 'dependents']

In [9]:
# params for Kfold CLf function
nsplits = 5
list_C = [1.0]

# Call Kfold CLF
scores = utils.kfold_biclf_logisticreg(df_train_full, 'churn', list_features, nsplits, list_C)

  0%|          | 0/1 [00:00<?, ?it/s]

C =1.0 score_mean=84.041, score_std=0.854


In [10]:
# Train final model 
y_train_full = df_train_full.churn.values
dv, model = utils.train_biclf_logisticreg(df_train_full, y_train_full, list_features, C=1.0)
y_pred = utils.predict_clf(df_test, list_features, dv, model)

# AUC final model
y_test = df_test.churn.values
auc = utils.get_roc_auc_score(y_test, y_pred)

auc

0.8572386167896259

***